# NLP Project

## Load Data

### Scraped CDP Scores
See [./download_csp_scores.py](./download_cdp_scores.py) for the CDP scores scraping code

In [ ]:
import pandas as pd
import numpy as np
import platform
from transformers import pipeline
import tensorflow as tf
import torch
from flax import nnx
pd.options.display.max_colwidth = 100

In [2]:
p = platform.platform()

esg_documents_df = pd.read_csv('./dax_esg_media_dataset/esg_documents_for_dax_companies.csv', sep='|')
cdp_scores_df = pd.read_csv('./cdp_scores.csv')
esg_documents_df = esg_documents_df.drop("Unnamed: 0", axis=1)



In [3]:
# merge CDP score with the esg data so that we have a score for each document
merged_df = pd.merge(esg_documents_df, cdp_scores_df, how='left', left_on='symbol', right_on='Ticker')

merged_df.dropna(subset=['CDP Score'], inplace=True) # drop rows without a CDP score

# print(merged_df[['symbol', 'CDP Score']])
# print(len(merged_df))
s = merged_df.head(1)['content'].item() # this is how to get the text alone

In [4]:
# big companies that I know
# Adidas, Volkswagen, Porsche, Siemens (AG and Energy), Merck, Airbus, Puma, Allianz, Brenntag
# this is all records for Adidas
tmp = merged_df[merged_df['company'] == "Adidas AG"]

## Preprocessing

In [ ]:
"""
    Company level
    CDP Score
    A      9
    A-    12
    B     10
    C      3

    Records for each company level
    CDP Score
    A     2760
    A-    4870
    B     2208
    C      360

Counts of how many companies have each type of score

C companies, arguably the most important, don't have as much to train on

another idea is to train on these German companies and then test on American companies (the extra data
thing that I have)

runs out of iterations because our training data is huge (several entire documents per company)
maybe we need to generate a equitable dataset, like equal amounts of labeled data
for each company that we have in the 36 we were able to get a CDP score on maybe we take 1200 samples
we can use the ClimateBERT to label them as positive, neutral, or negative and then create the training data
one record for each sentence/ phrase
maybe we can limit each phrase to a certain length or just separate by sentences or something

"""

# climateBERT model from https://huggingface.co/climatebert/environmental-claims?library=transformers
# environmental claims model - I wasn't sure which one to pick from their suite of models, this one semantically made the most sense to me
# there were a bunch of other ones

from transformers import AutoTokenizer, AutoModelForSequenceClassification

climate_tokenizer = AutoTokenizer.from_pretrained("climatebert/environmental-claims")
climate_model = AutoModelForSequenceClassification.from_pretrained("climatebert/environmental-claims") # this only gives yes/no
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert") # this gives positive/neutral/negative

message = "carbon emissions increased by about 100%"

example_content = tmp.head(1)['content'].item()
sentences = example_content.split(".")

examples = {

    "positive": [],
    "neutral": [], # with the ClimateBERT model, there is no neutral classification
    "negative": []
}

# inputs = climate_tokenizer(message, return_tensors="pt")
# with torch.no_grad():
#     logits = climate_model(**inputs).logits
# predicted_class_id = logits.argmax().item()
# if predicted_class_id == 0:
#     print("positive")
# else:
#     print("negative")

for s in sentences:
    if len(examples['positive']) > 1200:
        break
    try:
        inputs = climate_tokenizer(s, return_tensors="pt")
        with torch.no_grad():
            logits = climate_model(**inputs).logits
        predicted_class_id = logits.argmax().item()
    except Exception as e:
        print("did not work for this iteration of sentence, skipping")
        continue
    if predicted_class_id == 0:
        examples['positive'].append(s)
    else:
        examples['negative'].append(s)
    


Token indices sequence length is longer than the specified maximum sequence length for this model (663 > 512). Running this sequence through the model will result in indexing errors


did not work for this iteration of sentence, skipping
did not work for this iteration of sentence, skipping
did not work for this iteration of sentence, skipping
did not work for this iteration of sentence, skipping


In [ ]:
from pprint import pprint
pprint(examples['negative'])

[' In addition to the continued digital transformation of the company, we are '
 'working closely with our partners in the global supply chain to reduce '
 'energy consumption and to increase the proportion of green energy we use',
 ' By 2025, nine out of every ten adidas articles should be made from more '
 'sustainable materials, while the carbon footprint per product should be '
 'reduced by 15%',
 ' adidas aims at operating climate neutral (CO2e) at its locations by 2025 '
 'and at reaching complete climate neutrality by 2050',
 ' Our ambition is that 90% of our articles will be sustainable by 2025',
 ' The majority of the environmentally preferred materials currently used are '
 'recycled materials and more sustainable cotton',
 ' The percentages of the required proportion of environmentally preferred '
 'materials of the article weight have been increased significantly, which '
 'corresponds to our ambition to significantly expand our commitment to '
 'sustainability in the years

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    lowercase=True,
    stop_words='english',
    token_pattern=r'\b[a-zA-Z]{2,}\b', # Only keep word tokens with at least 2 letters
    max_features=100
)

bow_matrix = vectorizer.fit_transform(merged_df['content'].fillna(''))
y = merged_df['CDP Score']
# can use the bow_matrix in the next step for building a model

## Model Fitting

In [6]:
# https://www.geeksforgeeks.org/text-classification-using-logistic-regression/#
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

feature_names = vectorizer.get_feature_names_out()

# Print the first 10 features as an example
print(f"BoW features: [{feature_names}, ...]")

X_train, X_test, y_train, y_test = train_test_split(bow_matrix, y, test_size=0.25, random_state=1)


model = LogisticRegression(random_state=1, max_iter=1000000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:", confusion_matrix(y_test, y_pred))



BoW features: [['ag' 'al' 'analysis' 'annual' 'assets' 'based' 'billion' 'board'
 'business' 'cash' 'change' 'climate' 'com' 'companies' 'company'
 'compliance' 'consolidated' 'corporate' 'covid' 'current' 'customers'
 'data' 'development' 'emissions' 'employees' 'end' 'energy' 'et'
 'executive' 'financial' 'future' 'germany' 'global' 'google' 'group'
 'growth' 'health' 'high' 'human' 'impact' 'including' 'income' 'increase'
 'industry' 'information' 'key' 'level' 'like' 'long' 'management'
 'market' 'million' 'net' 'new' 'non' 'number' 'patients' 'people'
 'performance' 'period' 'power' 'product' 'production' 'products' 'pubmed'
 'rate' 'related' 'report' 'reporting' 'research' 'risk' 'risks' 'said'
 'sales' 'scholar' 'scopus' 'services' 'share' 'solutions' 'statements'
 'strategy' 'study' 'supervisory' 'supply' 'support' 'sustainability'
 'sustainable' 'systems' 'technology' 'term' 'time' 'total' 'use' 'used'
 'using' 'value' 'work' 'world' 'year' 'years'], ...]
Accuracy: 0.553333333

In [7]:
message = ""

vect_message = vectorizer.transform([message])
pred = model.predict(vect_message)
pred

array(['A-'], dtype=object)

In [ ]:
# TODO here we should try and build a model that correlates the preprocessed
# data to the column of our choice in the sp500_risk_ratings dataset


CountVectorizer(stop_words='english', token_pattern='\\b\\w+\\b')

In [ ]:
# this is the FinBERT model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# examples of text we can feed the model in order for it to classify it
# these examples are from the DAX dataset we are looking at
positive_example = "we have reduced our carbon emissions by 12.7% (vs. 2018)"
neutral_example = "We are proud of this result"
negative_example = "We have increased our carbon emissions by 12.7% (vs. 2018), which is a very bad thing"
false_positive = "we have increased our carbon emissions by 12.7% (vs. 2018)"

message = "we increased our carbon emissions by 400% this year!"


inputs = tokenizer(neutral_example, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'neutral'

In [ ]:
# climateBERT model from https://huggingface.co/climatebert/environmental-claims?library=transformers
# environmental claims model - I wasn't sure which one to pick from their suite of models, this one semantically made the most sense to me
# there were a bunch of other ones

from transformers import AutoTokenizer, AutoModelForSequenceClassification

climate_tokenizer = AutoTokenizer.from_pretrained("climatebert/environmental-claims")
climate_model = AutoModelForSequenceClassification.from_pretrained("climatebert/environmental-claims")

message = "we increased our carbon emissions by 400% this year!"

inputs = climate_tokenizer(neutral_example, return_tensors="pt")

with torch.no_grad():
    logits = climate_model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'positive'

as we can see here, the second model correctly predicts the false positive whereas the first model does not

## Model Analysis

In [ ]:
# TODO check how well our model did here

# Data Sources
- https://www.kaggle.com/datasets/pritish509/s-and-p-500-esg-risk-ratings
- https://www.kaggle.com/datasets/equintel/dax-esg-media-dataset

# References

# Scratch Work

In [ ]:
import pandas as pd
pd.read_csv("~/class/f24/nlp/project/sustain-nlp/sp500_esg_risk_ratings/sp500_esg_risk_ratings.csv")

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
0,ENPH,"Enphase Energy, Inc.","47281 Bayside Parkway\nFremont, CA 94538\nUnit...",Technology,Solar,"3,157","Enphase Energy, Inc., together with its subsid...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EMN,Eastman Chemical Company,"200 South Wilcox Drive\nKingsport, TN 37662\nU...",Basic Materials,Specialty Chemicals,"14,000",Eastman Chemical Company operates as a special...,25.3,12.8,6.6,5.8,Moderate Controversy Level,2.0,50th percentile,Medium
2,DPZ,Domino's Pizza Inc.,"30 Frank Lloyd Wright Drive\nAnn Arbor, MI 481...",Consumer Cyclical,Restaurants,"6,500","Domino's Pizza, Inc., through its subsidiaries...",29.2,10.6,6.3,12.2,Moderate Controversy Level,2.0,66th percentile,Medium
3,DAY,"Dayforce, Inc.","3311 East Old Shakopee Road\nMinneapolis, MN 5...",Technology,Software - Application,"9,084","Dayforce Inc., together with its subsidiaries,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DVA,Davita Inc.,"2000 16th Street\nDenver, CO 80202\nUnited States",Healthcare,Medical Care Facilities,"70,000",DaVita Inc. provides kidney dialysis services ...,22.6,0.1,8.4,14.1,Moderate Controversy Level,2.0,38th percentile,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,ACN,Accenture Plc,1 Grand Canal Square\nGrand Canal Harbour\nDub...,Technology,Information Technology Services,"742,000","Accenture plc, a professional services company...",9.8,0.8,4.4,4.6,Moderate Controversy Level,2.0,3rd percentile,Negligible
499,ABBV,Abbvie Inc.,"1 North Waukegan Road\nNorth Chicago, IL 60064...",Healthcare,Drug Manufacturers - General,"50,000","AbbVie Inc. discovers, develops, manufactures,...",29.9,2.4,10.4,17.2,Significant Controversy Level,3.0,69th percentile,Medium
500,ABT,Abbott Laboratories,100 Abbott Park Road\nAbbott Park\nNorth Chica...,Healthcare,Medical Devices,"114,000","Abbott Laboratories, together with its subsidi...",24.8,2.3,8.3,14.2,Significant Controversy Level,3.0,48th percentile,Medium
501,AOS,A.O. Smith Corporation,11270 West Park Place\nSuite 170 PO Box 245008...,Industrials,Specialty Industrial Machinery,"12,000",A. O. Smith Corporation manufactures and marke...,25.4,7.2,6.4,11.9,Low Controversy Level,1.0,51st percentile,Medium


In [ ]:
# if you want to find text data from S&P companies - this is the code to load it
# source: https://www.kaggle.com/datasets/jaidityachopra/esg-sustainability-reports-of-s-and-p-500-companies 
# pd.read_csv("~/class/f24/nlp/project/extra_data/preprocessed_content.csv").head(1)['preprocessed_content']

0    style guide colour colour use imagecolour prof...
Name: preprocessed_content, dtype: object